In [32]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [33]:
# Merging two tables together through joins
name = "LT Gas Data.xlsx"
supply = pd.read_excel(name,sheet_name=0)
demand = pd.read_excel(name,sheet_name=1)
template = pd.read_excel(name,sheet_name=2)
demand.index = demand["Product"]
demand.drop(["Product"],axis=1,inplace=True)
demand = demand.transpose()
demand.index = pd.to_datetime(demand.index)
demand["month"] = demand.index.month
supply["Delivery Date"] = pd.to_datetime(supply["Delivery Date"])
supply["month"] = supply["Delivery Date"].dt.month

merged = demand.merge(supply, on='month', how='left')
mapping = {
"Product A_x":"Demand A",
"Product B_x":"Demand B",
"Product C_x":"Demand C",
"Product D_x":"Demand D",
"Product A_y":"Supply A",
"Product B_y":"Supply B",
"Product C_y":"Supply C",
"Product D_y":"Supply D",
"month":"Month",
"Delivery Date":"Delivery Date (Daily)"}

merged.columns = merged.columns.map(mapping)
gb = merged.groupby('Month')    
lst = [gb.get_group(x) for x in gb.groups]

letters = ["A","B","C","D"]

gb_prod = []

for i in lst:

    for l in letters:
        cols = [i for i in list(mapping.values())[:-2] if l == i[-1]]
        total_cols = ["Month","Delivery Date (Daily)"] + cols
        df = i[total_cols]
        df["Product"] = l
        df.index = df["Delivery Date (Daily)"]
        df.drop(["Month","Delivery Date (Daily)"],axis=1,inplace=True)
        df.rename({f"Demand {l}": "Demand", f"Supply {l}": "Supply"}, axis=1, inplace=True)
        gb_prod.append(df)

all_df = pd.concat(gb_prod)
all_df.to_csv("PositionData.csv")

In [34]:
all_df

Demand  Supply Product
Delivery Date (Daily)                        
2023-03-01                 42      50       A
2023-03-02                 42      50       A
2023-03-03                 42      50       A
2023-03-04                 42      50       A
2023-03-05                 42      50       A
...                       ...     ...     ...
2023-12-27                 43      25       D
2023-12-28                 43      25       D
2023-12-29                 43      25       D
2023-12-30                 43      25       D
2023-12-31                 43      25       D

[1224 rows x 3 columns]

In [35]:
# Calculating aggregate positions for report
gb = all_df.groupby("Product")
gbs = [gb.get_group(x) for x in letters]
quarterly = []
monthly = []
for i in gbs:
    quarterly_positions = i.resample('Q').sum()
    quarterly_positions["Product"]= i["Product"][0]
    quarterly_positions.index = ["Q1","Q2","Q3","Q4"]
    
    monthly_positions = i.resample('M').sum()
    monthly_positions["Product"]= i["Product"][0]    
    months = ["March","April","May","June","July","August",
              "September","October","November","December"]
    monthly_positions.index = months
    
    quarterly.append(quarterly_positions)
    monthly.append(monthly_positions)
    
for i in monthly:
    display(i)
    
for j in quarterly:
    display(j)

Demand  Supply Product
March        1302    1550       A
April         930     450       A
May           558     465       A
June          300     450       A
July          248     465       A
August        279     465       A
September     360     450       A
October       496     775       A
November      690     750       A
December     1085     775       A

Demand  Supply Product
March        1953    2170       B
April        1380     600       B
May           837     620       B
June          450     600       B
July          372     620       B
August        403     620       B
September     540     600       B
October       713     930       B
November     1050     900       B
December     1612     930       B

Demand  Supply Product
March        2294    1860       C
April        1620    1200       C
May           992    1240       C
June          540    1200       C
July          434    1240       C
August        465    1240       C
September     630    1200       C
October       837    1085       C
November     1230    1050       C
December     1891    1085       C

Demand  Supply Product
March        1643    1240       D
April        1170     600       D
May           713     620       D
June          390     600       D
July          310     620       D
August        341     620       D
September     450     600       D
October       620     775       D
November      870     750       D
December     1333     775       D

Demand  Supply Product
Q1    1302    1550       A
Q2    1788    1365       A
Q3     887    1380       A
Q4    2271    2300       A

Demand  Supply Product
Q1    1953    2170       B
Q2    2667    1820       B
Q3    1315    1840       B
Q4    3375    2760       B

Demand  Supply Product
Q1    2294    1860       C
Q2    3152    3640       C
Q3    1529    3680       C
Q4    3958    3220       C

Demand  Supply Product
Q1    1643    1240       D
Q2    2273    1820       D
Q3    1101    1840       D
Q4    2823    2300       D